In [1]:
import pyspark

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Fraction Calculation") \
    .getOrCreate()

# Doc du lieu tu tep co san tren may
df = spark.read.text("data_phanso")

In [17]:
# tach tu so voi mau so
getvalue = df.selectExpr("split(value, '/') as fraction") \
                   .selectExpr("cast(fraction[0] as int) as numerator", "cast(fraction[1] as int) as denominator")

# tim boi so chung nho nhat cua mau so
# tim so chung lon nhat
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a
# tim boi so chung nho nhat
def lcm(a, b):
    return a * b / gcd(a, b)

# tinh boi so chung nho nhat cua mau so
bcnn = getvalue.select(expr("max(denominator)")).collect()[0][0]
for row in getvalue.collect():
    bcnn = lcm(bcnn, row["denominator"])

# tinh tong phan so
total_numerator = 0
for row in getvalue.collect():
    total_numerator += row["numerator"] * (bcnn // row["denominator"])

# rut gon phan so
gcd_value = gcd(total_numerator, bcnn)
total_numerator /= gcd_value
bcnn /= gcd_value

print("Tong phan so", int(total_numerator), "/", int(bcnn))

spark.stop()

Tong phan so 151 / 40
